# Calculating Zonal Statistics 
MS 263 Final Project

Caroline Daley | Moss Landing Marine Laboratories 

May 2025

In [42]:
import geopandas as gpd
import pandas as pd
from rasterstats import zonal_stats
import glob
import os
import xdem
#import re
#from rasterio.plot import show


In [53]:
from shapely.geometry import box

Connect to local database to access habitat metric files created in "", as well as a shapefile of CCFRP gridcells. Make sure the habitat metric rasters and gridcell shapefile are in the same spatial reference system. 

In [68]:
dem_folder = '/Users/carolinedaley/Documents/MLML/MS263/Project/Data/TRIAL'
shapefile_path = '/Users/carolinedaley/Documents/MLML/MS263/Project/Data/shapefiles/Grid Cells Shapefile 2021/CCFRP_Grid_Cells_2021.shp'

Make sure that the spatial reference of the shapefile matches the rasters of habitat metrics. 

In [69]:
import geopandas as gpd
import rasterio
import glob
import os

# --- Paths ---
reprojected_shapefile_path = '/Users/carolinedaley/Documents/MLML/MS263/Project/Data/shapefiles/Grid_Cells_2021_reprojected.shp'


# --- Read original shapefile ---
gridcells = gpd.read_file(shapefile_path)

# --- Get CRS from first DEM ---
dem_files = glob.glob(os.path.join(dem_folder, '*.tif'))
first_dem = dem_files[0]

# Get the CRS from the first DEM file
with rasterio.open(first_dem) as src:
    common_crs = src.crs  # This will be used consistently everywhere

# --- Reproject and save ---
gridcells = gridcells.to_crs(common_crs)
gridcells.to_file(reprojected_shapefile_path)
print(f"Reprojected shapefile saved to: {reprojected_shapefile_path}")


IndexError: list index out of range

In [67]:
# --- Set paths ---
shapefile_path = '/Users/carolinedaley/Documents/MLML/MS263/Project/Data/shapefiles/Grid_Cells_2021_reprojected.shp'
test_file = '/Users/carolinedaley/Documents/MLML/MS263/Project/Data/TRIAL/Bathymetry_OffshoreBodegaHead_rugosity.tif'

# --- Load reprojected shapefile ---
gdf = gpd.read_file(shapefile_path)

# --- Plot the DEM and shapefile ---
with rasterio.open(test_file) as src:
    dem_crs = src.crs

    # --- Plot ---
    fig, ax = plt.subplots(figsize=(10, 10))

    # Show DEM
    show(src, ax=ax, title="DEM with Reprojected Shapefile Overlay")

    # Show shapefile
    gdf.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2)

    # Focus on DEM extent
    ax.set_xlim(src.bounds[0], src.bounds[2])
    ax.set_ylim(src.bounds[1], src.bounds[3])

    plt.show()

print(shapefile.head())

RasterioIOError: /Users/carolinedaley/Documents/MLML/MS263/Project/Data/TRIAL/Bathymetry_OffshoreBodegaHead_rugosity.tif: No such file or directory

In [65]:
import numpy as np

# --- Initialize list for all results ---
all_stats = []

for dem_path in dem_files:
    with rasterio.open(dem_path) as src:
        dem_name = os.path.basename(dem_path)
        bounds = src.bounds

        # Check the nodata value in the DEM
        nodata_value = src.nodata
        print(f"Nodata value for {dem_name}: {nodata_value}")

        # --- Debugging step: Check raster data for valid values ---
        data = src.read(1)  # Read the first band of the raster
        unique_values = np.unique(data)  # Check unique values
        print(f"Unique values in the DEM: {unique_values[:10]}")  # Show first 10 unique values

        # Check if the data has any valid values
        valid_data = data[data != nodata_value]
        print(f"Valid data count: {len(valid_data)}")

        # Create bounding box GeoDataFrame for the DEM
        dem_bbox = gpd.GeoDataFrame(geometry=[box(*bounds)], crs=common_crs)

        # Filter grid cells that intersect with DEM bounding box
        intersecting_cells = gdf_all[gdf_all.intersects(dem_bbox.geometry.iloc[0])]

        if intersecting_cells.empty:
            print(f"No intersecting grid cells for: {dem_name}")
            continue
        
        # Debugging: Check how many grid cells are intersecting
        print(f"Number of intersecting grid cells for {dem_name}: {len(intersecting_cells)}")
        
        # Run zonal statistics
        stats = zonal_stats(
            intersecting_cells,
            dem_path,
            stats=["mean", "min", "max", "std"],
            nodata=nodata_value  # Use the same nodata value from the DEM
        )

        if not stats:
            print(f"No zonal stats calculated for {dem_name}, stats is empty.")
            continue

        # Combine stats with metadata
        temp_df = intersecting_cells.copy()
        temp_df[["mean", "min", "max", "std"]] = pd.DataFrame(stats)
        temp_df["dem_file"] = dem_name

        # No need to reproject again, as everything is in common_crs
        all_stats.append(temp_df)

# --- Combine all results ---
if all_stats:
    result = pd.concat(all_stats, ignore_index=True)
    result.to_csv("zonal_stats_by_DEM.csv", index=False)
    print("Saved zonal stats to zonal_stats_by_DEM.csv")
else:
    print("No overlapping stats computed.")



Nodata value for Bathymetry_OffshorePointConception_rugosity.tif: -3.4028234663852886e+38
Unique values in the DEM: [-3.4028235e+38  1.0000000e+00  1.0000001e+00  1.0000002e+00
  1.0000004e+00  1.0000005e+00  1.0000006e+00  1.0000007e+00
  1.0000008e+00  1.0000010e+00]
Valid data count: 134492005
Number of intersecting grid cells for Bathymetry_OffshorePointConception_rugosity.tif: 25
Nodata value for Bathymetry_OffshoreBodegaHead_rugosity.tif: -3.4028230607370965e+38
Unique values in the DEM: [-3.4028231e+38  1.0000000e+00  1.0000031e+00  1.0000062e+00
  1.0000094e+00  1.0000125e+00  1.0000156e+00  1.0000187e+00
  1.0000218e+00  1.0000219e+00]
Valid data count: 34873043
Number of intersecting grid cells for Bathymetry_OffshoreBodegaHead_rugosity.tif: 27
Nodata value for Bathymetry_OffshoreCapeMendocino_rugosity.tif: -3.4028234663852886e+38
Unique values in the DEM: [-3.4028235e+38  1.0000000e+00  1.0000031e+00  1.0000062e+00
  1.0000063e+00  1.0000093e+00  1.0000094e+00  1.0000124e+00

Min value in DEM: 1.0, Max value in DEM: 6.671098232269287
